In [5]:
'''
1. データセットの読み込みと前処理
'''
from tensorflow.keras import datasets

# Fashion-MNISTデータセットの読み込み
(x_train, t_train), (x_test, t_test) = datasets.fashion_mnist.load_data()

# 訓練データを正規化
x_train = x_train.astype('float32') / 255
# テストデータを正規化
x_test =  x_test.astype('float32') / 255

# (60000, 28, 28)の3階テンソルを(60000, 28, 28, 1)の4階テンソルに変換
x_train = x_train.reshape(-1, 28, 28, 1)
# (10000, 28, 28)の3階テンソルを(10000, 28, 28, 1)の4階テンソルに変換
x_test = x_test.reshape(-1, 28, 28, 1)

In [2]:
# 畳み込みネットワークの構築
from tensorflow.keras import models, layers, optimizers, regularizers

# tf.keras.models.Sequentialはtf.keras.Sequentialのエイリアス
model = models.Sequential()

# 正則化の係数
weight_decay = 1e-4

# （第1層）畳み込み層1
# ニューロン数：64
# 出力：1ニューロンあたり(28, 28, 1)の3階テンソルを64出力するので
#       (28, 28, 64)の出力となる
model.add(
    layers.Conv2D(
        filters=64,              # フィルターの数は64
        kernel_size=(3, 3),      # 3×3のフィルターを使用
        input_shape=(28, 28, 1), # 入力データのサイズ
        padding='same',          # ゼロパディングを行う
        kernel_regularizer=regularizers.l2(
            weight_decay),       # 正則化
        activation='relu'        # 活性化関数はReLU
        ))

# （第2層）畳み込み層2
# ニューロン数：32
# 出力：1ニューロンあたり(28, 28, 1)の3階テンソルを32出力するので
#       (28, 28, 32)の出力となる
model.add(
        layers.Conv2D(
        filters=32,              # フィルターの数は32
        kernel_size=(3, 3),      # 3×3のフィルターを使用
        padding='same',          # ゼロパディングを行う
        kernel_regularizer=regularizers.l2(
           weight_decay),       # 正則化
        activation='relu'        # 活性化関数はReLU
        ))

# （第3層）プーリング層1
# ニューロン数：32
# 出力：1ニューロンあたり(14, 14, 1)の3階テンソルを32出力するので
#       (14, 14, 32)の出力となる
model.add(
    layers.MaxPooling2D(
        pool_size=(2, 2)))       # 縮小対象の領域は2×2

# （第4層）畳み込み層3
# ニューロン数：16
# 出力：1ニューロンあたり(14 14 1)の3階テンソルを16出力するので
#       (14, 14, 16)の出力となる
model.add(
    layers.Conv2D(
        filters=16,              # フィルターの数は16
        kernel_size=(3, 3),      # 3×3のフィルターを使用
        padding='same',          # ゼロパディングを行う
        kernel_regularizer=regularizers.l2(
           weight_decay),       # 正則化
        activation='relu'        # 活性化関数はReLU
        ))

# （第5層）プーリング層2
# ニューロン数：16
# 出力：1ニューロンあたり(7, 7, 1)の3階テンソルを16出力するので
#       (7, 7, 16)の出力となる
model.add(
    layers.MaxPooling2D(
        pool_size=(2, 2)))       # 縮小対象の領域は2×2

# ドロップアウト40％
model.add(layers.Dropout(0.4))

# Flatten
# ニューロン数＝7×7×16=784
# (7, 7, 16)を(784)にフラット化
model.add(layers.Flatten())

# （第6層）全結合層
# ニューロン数：128
# 出力：(128)の1階テンソルを出力
model.add(
    layers.Dense(
        128,                     # ニューロン数は128
        activation='relu'))      # 活性化関数はReLU
        

# （第7層）出力層
# ニューロン数：10
# 出力：(10)の1階テンソルを出力
model.add(
    layers.Dense(
        10,                      # 出力層のニューロン数は10
        activation='softmax'))   # 活性化関数はソフトマックス

# 学習率
learning_rate = 0.1                  
# モデルのコンパイル
model.compile(
    # 損失関数はスパースラベル対応クロスエントロピー誤差
    loss='sparse_categorical_crossentropy',
    # オプティマイザーはSGD
    optimizer=optimizers.SGD(lr=learning_rate),
    # 学習評価として正解率を指定
    metrics=['accuracy'])

# サマリーを表示
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 16)        4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0

In [3]:
%%time
'''
3. モデルの学習
'''
# エポック数
epoch = 100
# ミニバッチのサイズ
batch_size = 64

history = model.fit(
    x_train,               # 訓練データ
    t_train,               # 正解ラベル
    batch_size=batch_size, # ミニバッチのサイズを設定
    epochs=epoch,          # エポック数を設定
    verbose=1,             # 進捗状況を出力する
    validation_split=0.2,  # 20パーセントのデータを検証に使用
    shuffle=True
    )

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 146s 3ms/sample - loss: 0.7032 - accuracy: 0.7425 - val_loss: 0.4334 - val_accuracy: 0.8396
Epoch 2/100
48000/48000 [==============================] - 142s 3ms/sample - loss: 0.4540 - accuracy: 0.8354 - val_loss: 0.3801 - val_accuracy: 0.8618
Epoch 3/100
48000/48000 [==============================] - 148s 3ms/sample - loss: 0.3987 - accuracy: 0.8570 - val_loss: 0.3344 - val_accuracy: 0.8788
Epoch 4/100
48000/48000 [==============================] - 139s 3ms/sample - loss: 0.3656 - accuracy: 0.8672 - val_loss: 0.3212 - val_accuracy: 0.8833
Epoch 5/100
48000/48000 [==============================] - 140s 3ms/sample - loss: 0.3432 - accuracy: 0.8762 - val_loss: 0.2946 - val_accuracy: 0.8963
Epoch 6/100
48000/48000 [==============================] - 142s 3ms/sample - loss: 0.3223 - accuracy: 0.8832 - val_loss: 0.2908 - val_accuracy: 0.8947
Epoch 7/100
48000/48000 [===================

In [6]:
'''
4. テストデータによるモデルの評価
'''
# 学習済みのモデルにテストデータを入力して損失と精度を取得
test_loss, test_acc = model.evaluate(x_test, t_test, verbose=0)
print('test_loss:', test_loss)
print('test_acc:', test_acc)

test_loss: 0.2571550129354
test_acc: 0.9271
